<a href="https://colab.research.google.com/github/HillaryKoros/Upwork_Work/blob/main/Crop_Data_Extraction_(161223).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1(Introduction)

## Crop Timeseries  Data Extraction per USA State and Upload to PostgreSQL
This notebook will Leverage the data from:

 [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL) a yearly land cover dataset tailored to crops, covering the entire continental United States. This dataset is generated using moderate-resolution satellite images and detailed agricultural ground information. The CDL is developed by the Spatial Analysis Research Section within the Geospatial Information Branch of the Research and Development Division at the USDA's National Agricultural Statistics Service (NASS).

[TIGER: US Census States 2018](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) an extracts of selected geographic and cartographic information from the U.S. Census Bureau's Master Address File/Topologically Integrated Geographic Encoding and Referencing (MAF/TIGER) database. The shapefiles include information for the fifty states, the District of Columbia, Puerto Rico, and the Island areas (American Samoa, the Commonwealth of the Northern Mariana Islands, Guam, and the United States Virgin Islands).

[FAQ USDA NASS Cropland Data Layers](https://www.nass.usda.gov/Research_and_Science/Cropland/sarsfaqs2.php)

[FAQ TIGER: US Census States 2018](https://www.census.gov/programs-surveys/geography/guidance/tiger-data-products-guide.html)


## Prerequisites
To use  [Geemap](https://geemap.org/) and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial).


##  Milestones
**Extract dataset and put into Postgres**
* Retrieve Cornfield Data and Consolidate per State.

                  Could be aggregated further to counties,Municipalities Special districts and point dataset etc.

                  Requires County,municipalities,special districts boundary dataset which i don't have access to at the moment.

* Export Data as CSV/Shapefile for Each State from 1997 to 2021.

**Connect Python to Postgres**
* Establish Connection to PostgreSQL.
* Upload Data and Formulate Schema in PostgreSQL.
* Execute Various Queries for Testing Purposes.

**Create Sample Analysis in Python**
* Explore the Top States in Corn Production.
* Examine States Showing Fluctuations in Corn Farming Over Time.

**Spatio temporal visualization in Python**
* Generate Map Slides Depicting Cornfield Time Series for the Specified Period.
* Create Charts Showing Cornfield Time Series Across States Over Time.
* Develop Maps Highlighting Losses and Gains of Cornfields across States.



## Install libraries

In [ ]:
## ********" Uncomment to  install "*****************

# !pip install geemap
# !pip install plotly
#!pip install pycrs



## Import libraries

In [346]:
import ee
import geemap as gee
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import pycrs

# Section 2 (Dataset Extraction)

##  Create an interactive Map

In [347]:
#### ********Initialize interactive map and center to USA *************
Map1 = gee.Map(center=[40, -100], zoom=4, height=600)
Map1.add_basemap('HYBRID')
Map1


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Load the TIGER: US Census States 2018

In [348]:
Map2 = gee.Map(center=[40, -100], zoom=4, height=600)
Map2.add_basemap('HYBRID')
states_shp = '/content/States_shapefile.shp'
states = gee.shp_to_ee(states_shp)
Map2.addLayer(states, {}, 'US States')
Map2

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Load USDA CropLand Dataset

In [349]:
USDA_Cropland_DataC = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date('1997-01-01', '2022-01-01')).select('cropland')
USDA_Cropland_DataC

### Check Labels

In [350]:
USDA_labels = list(gee.builtin_legends['USDA/NASS/CDL'].keys())
print(USDA_labels)

['1 Corn', '2 Cotton', '3 Rice', '4 Sorghum', '5 Soybeans', '6 Sunflower', '10 Peanuts', '11 Tobacco', '12 Sweet Corn', '13 Pop or Orn Corn', '14 Mint', '21 Barley', '22 Durum Wheat', '23 Spring Wheat', '24 Winter Wheat', '25 Other Small Grains', '26 Dbl Crop WinWht/Soybeans', '27 Rye', '28 Oats', '29 Millet', '30 Speltz', '31 Canola', '32 Flaxseed', '33 Safflower', '34 Rape Seed', '35 Mustard', '36 Alfalfa', '37 Other Hay/Non Alfalfa', '38 Camelina', '39 Buckwheat', '41 Sugarbeets', '42 Dry Beans', '43 Potatoes', '44 Other Crops', '45 Sugarcane', '46 Sweet Potatoes', '47 Misc Vegs & Fruits', '48 Watermelons', '49 Onions', '50 Cucumbers', '51 Chick Peas', '52 Lentils', '53 Peas', '54 Tomatoes', '55 Caneberries', '56 Hops', '57 Herbs', '58 Clover/Wildflowers', '59 Sod/Grass Seed', '60 Switchgrass', '61 Fallow/Idle Cropland', '63 Forest', '64 Shrubland', '65 Barren', '66 Cherries', '67 Peaches', '68 Apples', '69 Grapes', '70 Christmas Trees', '71 Other Tree Crops', '72 Citrus', '74 Pecan

## Get Cropland Extent for 2021

In [351]:
USDA_Cropland_Data_2021 = USDA_Cropland_DataC.filterDate('2021-01-01', '2021-12-31').mosaic()
USDA_Cropland_Data_2021

In [352]:
legends = gee.builtin_legends
for legend in legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


### Visualize 2021 Cropland Extent

In [353]:
Map3= gee.Map(center=[40, -100], zoom=4, height=600)
Map3.add_basemap('HYBRID')

Map3.addLayer(USDA_Cropland_Data_2021.randomVisualizer(), {},'Crop Landcover')
Map3.addLayer(states, {}, 'States')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

 ## Get CornFields  timeseries



In [354]:
USA_cornland_col = USDA_Cropland_DataC.map(lambda img: img.eq(1).selfMask())
USA_cornland_ts = USA_cornland_col.toBands()
USA_cornland_ts

#### Get Image timeseries dates

In [355]:
years= (USA_cornland_col.aggregate_array('system:index').getInfo())
years

['1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005a',
 '2005b',
 '2006',
 '2007a',
 '2007b',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

## Zonal Statistics for each State
  

In [ ]:
gee.zonal_stats(
    USA_cornland_col,states, 'USA_cornland_perstate1997_2021.csv', statistics_type='SUM', scale=30
)

## **Clean and process the CSV dataset**

In [356]:
#3 Convert CSV to df
df = gee.csv_to_df('USA_cornland_perstate1997_2021.csv')
df.head(10)


,1997_cropland,1998_cropland,1999_cropland,2000_cropland,2001_cropland,2002_cropland,2003_cropland,2004_cropland,2005a_cropland,2005b_cropland,...,2019_cropland,2020_cropland,2021_cropland,system:index,FID,State_Code,Program,State_Name,Flowing_St,FID_1
0,0.0,0.0,0.000000,0.00000,0.552941,0.372549,0.552941,0.329412,0.000000,0.000000,...,1238.337255,1313.780392,1542.011765,0,1,AL,PERMIT TRACKING,ALABAMA,F,919
1,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1,2,AK,NaN,ALASKA,N,920
2,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,198.984314,211.925490,189.788235,2,3,AZ,AZURITE,ARIZONA,F,921
3,0.0,0.0,90.941176,102.05098,29.000000,644.568627,748.592157,471.329412,499.905882,3.662745,...,2662.615686,2175.541176,2984.443137,3,4,AR,PDS,ARKANSAS,F,922
4,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,919.341176,690.078431,646.988235,4,5,CA,NaN,CALIFORNIA,N,923
5,0.0,0.0,0.000000,0.00000,0.000000,16.164706,14.227451,20.341176,14.188235,0.000000,...,6903.890196,6634.486275,6197.160784,5,6,CO,ECOMAP,COLORADO,F,924
6,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,78.384314,84.137255,85.666667,6,7,CT,SIMS,CONNECTICUT,F,925
7,0.0,0.0,0.000000,0.00000,0.000000,585.792157,0.000000,0.000000,0.000000,0.000000,...,1007.694118,963.090196,967.878431,7,8,DE,NaN,DELAWARE,P,926
8,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,8,9,DC,NaN,DISTRICT OF COLUMBIA,P,927
9,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,142.207843,0.000000,0.000000,...,426.780392,272.725490,319.443137,9,10,FL,NaN,FLORIDA,P,928


In [357]:
# check df columns
df.columns

Index(['1997_cropland', '1998_cropland', '1999_cropland', '2000_cropland',
       '2001_cropland', '2002_cropland', '2003_cropland', '2004_cropland',
       '2005a_cropland', '2005b_cropland', '2006_cropland', '2007a_cropland',
       '2007b_cropland', '2008_cropland', '2009_cropland', '2010_cropland',
       '2011_cropland', '2012_cropland', '2013_cropland', '2014_cropland',
       '2015_cropland', '2016_cropland', '2017_cropland', '2018_cropland',
       '2019_cropland', '2020_cropland', '2021_cropland', 'system:index',
       'FID', 'State_Code', 'Program', 'State_Name', 'Flowing_St', 'FID_1'],
      dtype='object')

In [358]:
# Remove '_cropland'
cropland_cols = [col for col in df.columns if col.endswith('_cropland')]
new_column_names = {col: col.replace('_cropland', '') for col in cropland_cols}
df.rename(columns=new_column_names, inplace=True)

df.head()

,1997,1998,1999,2000,2001,2002,2003,2004,2005a,2005b,...,2019,2020,2021,system:index,FID,State_Code,Program,State_Name,Flowing_St,FID_1
0,0.0,0.0,0.000000,0.00000,0.552941,0.372549,0.552941,0.329412,0.000000,0.000000,...,1238.337255,1313.780392,1542.011765,0,1,AL,PERMIT TRACKING,ALABAMA,F,919
1,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1,2,AK,NaN,ALASKA,N,920
2,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,198.984314,211.925490,189.788235,2,3,AZ,AZURITE,ARIZONA,F,921
3,0.0,0.0,90.941176,102.05098,29.000000,644.568627,748.592157,471.329412,499.905882,3.662745,...,2662.615686,2175.541176,2984.443137,3,4,AR,PDS,ARKANSAS,F,922
4,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,919.341176,690.078431,646.988235,4,5,CA,NaN,CALIFORNIA,N,923


In [359]:
df.columns


Index(['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005a',
       '2005b', '2006', '2007a', '2007b', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', 'system:index', 'FID', 'State_Code', 'Program', 'State_Name',
       'Flowing_St', 'FID_1'],
      dtype='object')

In [360]:
# Columns to remove
columns_to_remove = ['Flowing_St', 'FID_1','system:index']

# Drop the specified columns
df = df.drop(columns=columns_to_remove)
df.tail()

,1997,1998,1999,2000,2001,2002,2003,2004,2005a,2005b,...,2016,2017,2018,2019,2020,2021,FID,State_Code,Program,State_Name
46,0.0,0.0,0.000000,0.000000,0.000000,1276.396078,0.000000,0.000000,0.000000,0.0,...,1721.572549,1796.286275,1793.247059,2026.643137,2107.039216,1917.411765,47,VA,NaN,VIRGINIA
47,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1108.960784,1179.619608,1210.807843,966.070588,1027.431373,1096.000000,48,WA,WA-FSIS,WASHINGTON
48,0.0,0.0,0.000000,0.000000,0.000000,94.270588,0.000000,0.000000,0.000000,0.0,...,151.572549,146.584314,134.725490,161.701961,157.298039,160.913725,49,WV,NaN,WEST VIRGINIA
49,0.0,0.0,18.160784,41.843137,52.164706,32.227451,8612.717647,15276.113725,8969.133333,0.0,...,19600.862745,18933.219608,19252.494118,20939.313725,22708.521569,21800.611765,50,WI,WI-ESR,WISCONSIN
50,0.0,0.0,0.000000,0.000000,0.000000,4.988235,4.713725,3.584314,3.709804,0.0,...,325.156863,288.796078,319.458824,307.576471,326.423529,350.450980,51,WY,NaN,WYOMING


In [361]:
# Columns to round off to 3 decimal places
columns_to_round = [col for col in df.columns if col[:4] in ['1997', '1998', '1999', '2000', '2001', '2002', '2003',
                                                             '2004', '2005a', '2005b', '2006', '2007a', '2007b', '2008',
                                                             '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
                                                             '2017', '2018', '2019', '2020']]

for col in columns_to_round:
    df[col] = df[col].round(3)

df.tail()

,1997,1998,1999,2000,2001,2002,2003,2004,2005a,2005b,...,2016,2017,2018,2019,2020,2021,FID,State_Code,Program,State_Name
46,0.0,0.0,0.000,0.000,0.000,1276.396,0.000,0.000,0.000000,0.0,...,1721.573,1796.286,1793.247,2026.643,2107.039,1917.411765,47,VA,NaN,VIRGINIA
47,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000000,0.0,...,1108.961,1179.620,1210.808,966.071,1027.431,1096.000000,48,WA,WA-FSIS,WASHINGTON
48,0.0,0.0,0.000,0.000,0.000,94.271,0.000,0.000,0.000000,0.0,...,151.573,146.584,134.725,161.702,157.298,160.913725,49,WV,NaN,WEST VIRGINIA
49,0.0,0.0,18.161,41.843,52.165,32.227,8612.718,15276.114,8969.133333,0.0,...,19600.863,18933.220,19252.494,20939.314,22708.522,21800.611765,50,WI,WI-ESR,WISCONSIN
50,0.0,0.0,0.000,0.000,0.000,4.988,4.714,3.584,3.709804,0.0,...,325.157,288.796,319.459,307.576,326.424,350.450980,51,WY,NaN,WYOMING


## Export to local directory/Drive

In [340]:
# Export DataFrame to a CSV file
df.to_csv('USA_cornland_perstate1997_2021_formatted_notcleaned.csv', index=False)

## **Sample Plots**

In [362]:
# Sort the DataFrame by 'NAME' in ascending order
df_sorted = df.sort_values(by='State_Name', ascending=False)

In [363]:
df_sorted.head()

,1997,1998,1999,2000,2001,2002,2003,2004,2005a,2005b,...,2016,2017,2018,2019,2020,2021,FID,State_Code,Program,State_Name
50,0.0,0.0,0.000,0.000,0.000,4.988,4.714,3.584,3.709804,0.0,...,325.157,288.796,319.459,307.576,326.424,350.450980,51,WY,NaN,WYOMING
49,0.0,0.0,18.161,41.843,52.165,32.227,8612.718,15276.114,8969.133333,0.0,...,19600.863,18933.220,19252.494,20939.314,22708.522,21800.611765,50,WI,WI-ESR,WISCONSIN
48,0.0,0.0,0.000,0.000,0.000,94.271,0.000,0.000,0.000000,0.0,...,151.573,146.584,134.725,161.702,157.298,160.913725,49,WV,NaN,WEST VIRGINIA
47,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000000,0.0,...,1108.961,1179.620,1210.808,966.071,1027.431,1096.000000,48,WA,WA-FSIS,WASHINGTON
46,0.0,0.0,0.000,0.000,0.000,1276.396,0.000,0.000,0.000000,0.0,...,1721.573,1796.286,1793.247,2026.643,2107.039,1917.411765,47,VA,NaN,VIRGINIA


In [378]:
gee.bar_chart(df_sorted, x='State_Name', y=years,sort_column='2020',
              max_rows=5,title='Top 5 Corn farming States across the period')

# Section 3 (Connect to PostgresSQL)

### Local Connection

In [ ]:
### Install libraires needed for connections
# !pip install ipython-sql
# !pip install sqlalchemy
# !pip install psycopg2

In [ ]:
%load_ext sql

In [ ]:
from sqlalchemy import create_engine
import os

In [ ]:
host = "localhost"
database = "USDACorn_DB"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [ ]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [ ]:
connection_string

In [ ]:
%sql $connection_string

### Heroku Deployment

In [ ]:
from sqlalchemy import create_engine


host = "database ip in heroku"
database = "DB_Name"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')


In [ ]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"
# Create an engine to connect to the database
engine = create_engine(connection_string)
connection = engine.connect()
connection

# Section 4 ( Sample Analysis)

# Section 5 ( Visualization)